기존 은서 언니 결과에서 바뀐거

- `final_data 해원.xlsx` : 노래 DB af기반 감정 확률 벡터 추출 결과 부분 수정됨
- `top_five_df_2` 분위기가 유사한 노래 추천 결과 달라짐, 내용 유사 곡 추천 결과는 동일

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import islice

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity

path = '경로 지정'

In [2]:
data = pd.read_excel(path + '/최종 모델링/최종 파일 (해원 수정 후)/final_data 해원.xlsx',index_col=0)
print(data.shape)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/82109/Desktop/deep daiv. 프로젝트/최종 모델링/최종 파일 (해원 수정 후)/final_data 해원.xlsx'

# API에서 도서 정보 들고오기

In [2]:
import requests
import json

In [3]:
rest_api_key = "41d651c93152d5ec054dc828cacfa671"
url = "https://dapi.kakao.com/v3/search/book"
header = {"authorization": "KakaoAK "+rest_api_key}

In [6]:
#궁금한 도서의 isbn 입력
querynum = {"query": '참을 수 없는 존재의 가벼움'}

In [7]:
#도서 정보 불러오기
response = requests.get(url, headers=header, params = querynum)
content = response.text
책정보 = json.loads(content)['documents'][0]

In [8]:
책정보

{'authors': ['밀란 쿤데라'],
 'contents': '살아 있는 신화가 된 작가 밀란 쿤데라의 대표작 『참을 수 없는 존재의 가벼움』을 밀란 쿤데라가 직접 그린 일러스트를 바탕으로 디자인한 신선한 표지와 장정으로 새롭게 만나볼 수 있다. 역사에서 태어났으되, 역사를 뛰어넘는 인간의 실존 그 자체를 다루고 있는 작품으로, 1960년대 체코와 1970년대 유럽을 뒤흔들어 놓은 무거운 역사의 상처와 개인적 트라우마를 어깨에 짊어진 네 남녀의 생과 사랑의 모습을 그리고 있다.  고향의 작은 술집에서 일하며 근근이',
 'datetime': '2018-06-20T00:00:00.000+09:00',
 'isbn': '8937437562 9788937437564',
 'price': 17000,
 'publisher': '민음사',
 'sale_price': 13500,
 'status': '정상판매',
 'thumbnail': 'https://search1.kakaocdn.net/thumb/R120x174.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flbook%2Fimage%2F540501%3Ftimestamp%3D20240110152752',
 'title': '참을 수 없는 존재의 가벼움',
 'translators': ['이재룡'],
 'url': 'https://search.daum.net/search?w=bookpage&bookId=540501&q=%EC%B0%B8%EC%9D%84+%EC%88%98+%EC%97%86%EB%8A%94+%EC%A1%B4%EC%9E%AC%EC%9D%98+%EA%B0%80%EB%B2%BC%EC%9B%80'}

In [9]:
책정보['url']

'https://search.daum.net/search?w=bookpage&bookId=540501&q=%EC%B0%B8%EC%9D%84+%EC%88%98+%EC%97%86%EB%8A%94+%EC%A1%B4%EC%9E%AC%EC%9D%98+%EA%B0%80%EB%B2%BC%EC%9B%80'

In [10]:
book = pd.DataFrame({'title': 책정보['title'],
              'isbn': 책정보['isbn'],
              'authors': 책정보['authors'],
              'publisher': 책정보['publisher']})
book

,title,isbn,authors,publisher
0,참을 수 없는 존재의 가벼움,8937437562 9788937437564,밀란 쿤데라,민음사


In [11]:
title = book['title']
title[0]

'참을 수 없는 존재의 가벼움'

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

In [13]:
target_url = 책정보['url']
# 옵션 생성
options = webdriver.ChromeOptions()
# 창 숨기는 옵션 추가
options.add_argument("headless")

driver = webdriver.Chrome(options=options)
driver.get(target_url)
time.sleep(5)

try :
    botton = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[2]/div[3]/a')
    botton.click()
except :
    pass
책소개 = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[2]/p')

time.sleep(3)
try :
    botton = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[5]/div[3]/a')
    botton.click()
except :
    pass
책속으로 = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[5]/p')

time.sleep(3)
try :
    botton = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[6]/div[3]/a')
    botton.click()
except :
    pass
서평 = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[6]/p')

book['책소개'] = 책소개.text
book['책속으로'] = 책속으로.text
book['서평'] = 서평.text


In [14]:
img= driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[1]/div[1]/span/img')

In [15]:
img_src = img.get_attribute('src')
img_src

'https://search1.kakaocdn.net/thumb/C116x164.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flbook%2Fimage%2F540501%3Ftimestamp%3D20240110152752%3Fmoddttm=202401181445'

In [16]:
driver.close()

In [17]:
book

,title,isbn,authors,publisher,책소개,책속으로,서평
0,참을 수 없는 존재의 가벼움,8937437562 9788937437564,밀란 쿤데라,민음사,살아 있는 신화가 된 작가 밀란 쿤데라의 대표작 『참을 수 없는 존재의 가벼움』을 ...,그는 그녀에 대해서 거의 아무것도 알지 못하면서도 무어라 형언할 수 없는 사랑을 느...,“그들은 서로 사랑했는데도\n상대방에게 하나의 지옥을 선사했다.”\n\n매해 노벨 ...


# 도서 설명 text 전처리

In [18]:
import googletrans
from googletrans import Translator

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer

import time

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
#영어 불용어 사전
stops = set(stopwords.words('english'))

def hapus_url(text):
    mention_pattern = r'@[\w]+'
    cleaned_text = re.sub(mention_pattern, '', text)
    return re.sub(r'http\S+','', cleaned_text)

#특수문자 제거
#영어 대소문자, 숫자, 공백문자(스페이스, 탭, 줄바꿈 등) 아닌 문자들 제거
def remove_special_characters(text, remove_digits=True):
    text=re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text


#불용어 제거
def delete_stops(text):
    text = text.lower().split()
    text = ' '.join([word for word in text if word not in stops])
    return text
   
    
#품사 tag 매칭용 함수
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    

#품사 태깅 + 표제어 추출
def tockenize(text):
    tokens=word_tokenize(text)
    pos_tokens=nltk.pos_tag(tokens)
    
    text_t=list()
    for _ in pos_tokens:
        text_t.append([_[0], get_wordnet_pos(_[1])])
    
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word[0], word[1]) for word in text_t])
    return text



def clean(text):
    text = remove_special_characters(text, remove_digits=True)
    text = delete_stops(text)
    text = tockenize(text)
    return text

In [20]:
translator = Translator()
for col in ['책소개', '책속으로', '서평']:
    name = col+'_trans'
    if book[col].values == '':
        book[name] = ''
        continue
    time.sleep(1)
    book[name] = clean(translator.translate(hapus_url(book.loc[0, col])).text)

book

,title,isbn,authors,publisher,책소개,책속으로,서평,책소개_trans,책속으로_trans,서평_trans
0,참을 수 없는 존재의 가벼움,8937437562 9788937437564,밀란 쿤데라,민음사,살아 있는 신화가 된 작가 밀란 쿤데라의 대표작 『참을 수 없는 존재의 가벼움』을 ...,그는 그녀에 대해서 거의 아무것도 알지 못하면서도 무어라 형언할 수 없는 사랑을 느...,“그들은 서로 사랑했는데도\n상대방에게 하나의 지옥을 선사했다.”\n\n매해 노벨 ...,fresh cover manner design base illustration mi...,felt almost nothing felt unpelledshe look like...,even though love give hell person writer list ...


In [35]:
print(len(book['책소개'][0]))
print(book['책소개'][0])

842
살아 있는 신화가 된 작가 밀란 쿤데라의 대표작 『참을 수 없는 존재의 가벼움』을 밀란 쿤데라가 직접 그린 일러스트를 바탕으로 디자인한 신선한 표지와 장정으로 새롭게 만나볼 수 있다. 역사에서 태어났으되, 역사를 뛰어넘는 인간의 실존 그 자체를 다루고 있는 작품으로, 1960년대 체코와 1970년대 유럽을 뒤흔들어 놓은 무거운 역사의 상처와 개인적 트라우마를 어깨에 짊어진 네 남녀의 생과 사랑의 모습을 그리고 있다.

고향의 작은 술집에서 일하며 근근이 살던 젊은 테레자는 출장으로 그 도시에 들른 외과의사 토마시와 우연히 만난다. 전처와의 이혼 이후 진지한 사랑을 부담스러워하던 토마시는 강물에 떠내려 온 아기 같은 테레자의 연약한 매력을 놓지 못하고 고아를 떠맡듯 그녀와 함께 살기 시작한다. 하지만 스스로가 에로틱한 우정이라고 이름 붙인 그 가벼운 삶을 토마시는 버리지 못하고 이 여자 저 여자를 전전한다. 그런 토마시를 지켜보는 테레자는 질투와 체념으로 인한 괴로움에 몸부림친다.

소련의 침공으로 체코가 자유를 잃은 후, 두 사람은 함께 스위스로 넘어간다. 체코를 벗어나면 토마시의 연인들로부터도 벗어날 수 있을 거라 믿었던 테레자는 토마시의 끊임없는 외도에 믿음을 잃은 후 홀로 국경을 넘어 프라하로 돌아간다. 질투와 미움이 뒤섞인 두 사람의 삶은 그렇게 점차 무게를 더해 간다. 한편 토마시의 또 다른 연인이자 화가인 사비나는 끈질기게 자신을 따라다니는 조국과 역사의 무게에서 벗어나 자유롭게 살고 싶어 한다. 사비나는 체코에서 멀리, 할 수 있는 한 가장 멀리 떠나고, 사비나를 사랑하는 학자이자 한 가정의 가장으로서 안정된 일상을 누리던 프란츠는 그런 사비나의 가벼움에 매료되는데…….


In [21]:
total_text = book.loc[0, '책소개_trans'] + book.loc[0, '책속으로_trans'] + book.loc[0, '서평_trans']
total_text

'fresh cover manner design base illustration milan kunderas representative work milan kundera live mythborn history deal existence human being surpass history life love four men woman carry czech republic 1960s heavy history personal trauma heavy history shake europe 1970sis draw young teresa work small bar hometown meet tomasi surgeon visit citytomasi burden serious love since divorce ex wife begin live orphan without leave weak charm baby like babyhowever tomas cant throw away light life call erotic friendshipteresa watch tomas struggle pain jealousy resignation czech lose freedom due soviet invasion two go switzerland togetherteresa believe would able escape lover tomasi leave czech republic lose faith tomas constant affair return prague alonethe life two people mixed jealousy hatred gradually add weightmeanwhile tomasis another lover painter sabina want live freely weight homeland historysabina far czech republic farthest enjoy stable daily life love scholar love sabina family head

# 도서 설명 text 감정 벡터 추출

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

df = pd.read_csv(path + '/최종 모델링/최종 파일 (해원 수정 후)/tweet_data_agumentation.csv', index_col = 0)

tfidf_vect_emo = TfidfVectorizer()
tfidf_vect_emo.fit_transform(df["content"])

model = joblib.load(path + '/최종 모델링/최종 파일 (해원 수정 후)/SVM.pkl')
total_text2 = tfidf_vect_emo.transform(pd.Series(total_text))
model.predict_proba(total_text2)
sentiment = pd.DataFrame(model.predict_proba(total_text2),index=['prob']).T
sentiment['감정'] = ['empty','sadness','enthusiasm','worry','love','fun','hate','happiness','boredom','relief','anger']
sentiment_sorted = sentiment.sort_values(by='prob',ascending=False)
sentiment_sorted
sentiment
# sentiment 이용!

,prob,감정
0,0.000408,empty
1,0.062702,sadness
2,0.003605,enthusiasm
3,0.058310,worry
4,0.794448,love
5,0.002706,fun
6,0.000799,hate
7,0.060146,happiness
8,0.000238,boredom
9,0.016465,relief


In [22]:
target_url 

'https://search.daum.net/search?w=bookpage&bookId=540501&q=%EC%B0%B8%EC%9D%84+%EC%88%98+%EC%97%86%EB%8A%94+%EC%A1%B4%EC%9E%AC%EC%9D%98+%EA%B0%80%EB%B2%BC%EC%9B%80'

# audio feature 감정 벡터 - 도서 text 감정 벡터

In [23]:
audio_data = data.iloc[:,-11:]
audio_data

,empty_a,sadness_a,enthusiasm_a,worry_a,love_a,fun_a,hate_a,happiness_a,boredom_a,relief_a,anger_a
0,0.001346,0.001468,0.000573,0.999999,0.000172,0.000438,0.000084,0.000297,0.000171,0.000331,0.000293
1,0.000675,0.000728,0.000677,0.000237,0.001199,0.001109,0.000269,0.000683,0.999826,0.000080,0.000237
2,0.000721,0.000805,0.000329,0.000452,0.000202,0.000271,0.000053,0.000246,0.000452,0.999978,0.000072
3,0.002910,0.002584,0.000131,0.000329,0.002033,0.000132,0.000044,0.000142,0.000081,0.999633,0.000056
4,0.000334,0.000332,0.000241,0.000060,0.000185,0.000300,1.000000,0.000386,0.000421,0.000141,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
8515,1.000000,1.000000,0.000819,0.000420,0.999982,0.000613,0.000018,0.001577,0.000052,0.000307,0.000015
8516,0.016057,0.016190,0.000213,0.000669,0.022570,0.000219,0.992936,0.000332,0.000298,0.000344,0.992026
8517,0.999999,0.999998,0.000799,0.000706,0.999953,0.000468,0.000105,0.000734,0.000042,0.000216,0.000202
8518,0.999999,0.999999,0.000611,0.000276,0.999973,0.000354,0.000063,0.000703,0.000031,0.000216,0.000094


In [24]:
sentiment

,prob,감정
0,0.000408,empty
1,0.062702,sadness
2,0.003605,enthusiasm
3,0.058310,worry
4,0.794448,love
5,0.002706,fun
6,0.000799,hate
7,0.060146,happiness
8,0.000238,boredom
9,0.016465,relief


In [25]:
sentiment_prob = sentiment['prob']
sentiment_prob.index = sentiment['감정']
sentiment_prob

감정
empty         0.000408
sadness       0.062702
enthusiasm    0.003605
worry         0.058310
love          0.794448
fun           0.002706
hate          0.000799
happiness     0.060146
boredom       0.000238
relief        0.016465
anger         0.000174
Name: prob, dtype: float64

In [26]:
audio_data.columns = ['empty', 'sadness', 'enthusiasm', 'worry', 'love', 'fun', 'hate',
       'happiness', 'boredom', 'relief', 'anger']

In [27]:
# 도서 감정 벡터를, af 감정 벡터 데이터프레임에 이어붙히기
audio_data_1 = pd.concat([sentiment_prob, audio_data.T],axis=1).T
audio_data_1

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
prob,0.000408,0.062702,0.003605,0.058310,0.794448,0.002706,0.000799,0.060146,0.000238,0.016465,0.000174
0,0.001346,0.001468,0.000573,0.999999,0.000172,0.000438,0.000084,0.000297,0.000171,0.000331,0.000293
1,0.000675,0.000728,0.000677,0.000237,0.001199,0.001109,0.000269,0.000683,0.999826,0.000080,0.000237
2,0.000721,0.000805,0.000329,0.000452,0.000202,0.000271,0.000053,0.000246,0.000452,0.999978,0.000072
3,0.002910,0.002584,0.000131,0.000329,0.002033,0.000132,0.000044,0.000142,0.000081,0.999633,0.000056
...,...,...,...,...,...,...,...,...,...,...,...
8515,1.000000,1.000000,0.000819,0.000420,0.999982,0.000613,0.000018,0.001577,0.000052,0.000307,0.000015
8516,0.016057,0.016190,0.000213,0.000669,0.022570,0.000219,0.992936,0.000332,0.000298,0.000344,0.992026
8517,0.999999,0.999998,0.000799,0.000706,0.999953,0.000468,0.000105,0.000734,0.000042,0.000216,0.000202
8518,0.999999,0.999999,0.000611,0.000276,0.999973,0.000354,0.000063,0.000703,0.000031,0.000216,0.000094


In [28]:
col = ['book']+list(data['name'])

In [29]:
cosine_sim_audio = cosine_similarity(audio_data_1)
cosine_sim_audio_df = pd.DataFrame(cosine_sim_audio, index = col, columns=col)
cosine_sim_audio_df

,book,Still With You,death bed (coffee for your head),Nan-Nan,Mo-Eh-Wa,YASASHISA,Kiri Ga Naikara,Tsumi No Kaori,Cho Si Noccha Te,Tokuninai,...,My Heart Goes (La Di Da),Back To You,Tick Tack - Extended Mix,Turn Me Up - ViP Mix,Miserable Man,Murder in the City,September Song - Guitar Acoustic,Control,Heart's Content - Strings Mix,We Don't Have To Take Our Clothes Off
book,1.000000,0.073071,0.001620,0.020860,0.022798,0.001034,0.537115,0.537185,0.537131,0.020661,...,0.537184,0.072996,0.537129,0.537090,0.617767,0.617823,0.017760,0.617787,0.617774,0.004434
Still With You,0.073071,1.000000,0.000411,0.000786,0.000669,0.000310,0.000782,0.000806,0.000823,0.000699,...,0.000938,0.999998,0.000894,0.000807,0.002575,0.001967,0.000779,0.002132,0.001884,0.000344
death bed (coffee for your head),0.001620,0.000411,1.000000,0.000534,0.000168,0.000656,0.001918,0.001980,0.001887,0.000236,...,0.001913,0.000431,0.001903,0.001868,0.001568,0.001534,0.000606,0.001528,0.001521,0.999985
Nan-Nan,0.020860,0.000786,0.000534,1.000000,0.999994,0.000189,0.000556,0.000536,0.000637,1.000000,...,0.000732,0.000816,0.000537,0.000708,0.001472,0.001175,0.000355,0.001123,0.001123,0.001880
Mo-Eh-Wa,0.022798,0.000669,0.000168,0.999994,1.000000,0.000172,0.001254,0.001237,0.001335,0.999993,...,0.001432,0.000700,0.001235,0.001405,0.004821,0.004525,0.000412,0.004472,0.004472,0.001517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Murder in the City,0.617823,0.001967,0.001534,0.001175,0.004525,0.000361,0.290202,0.291140,0.290396,0.000810,...,0.290905,0.002087,0.290283,0.289860,1.000000,1.000000,0.022557,1.000000,1.000000,0.003102
September Song - Guitar Acoustic,0.017760,0.000779,0.000606,0.000355,0.000412,0.999746,0.008436,0.008540,0.008379,0.000314,...,0.008706,0.002296,0.008424,0.008415,0.022651,0.022557,1.000000,0.022668,0.022607,0.000346
Control,0.617787,0.002132,0.001528,0.001123,0.004472,0.000473,0.289906,0.290844,0.290099,0.000757,...,0.290609,0.002253,0.289986,0.289564,1.000000,1.000000,0.022668,1.000000,1.000000,0.003093
Heart's Content - Strings Mix,0.617774,0.001884,0.001521,0.001123,0.004472,0.000412,0.289814,0.290752,0.290007,0.000757,...,0.290517,0.002004,0.289894,0.289472,1.000000,1.000000,0.022607,1.000000,1.000000,0.003087


In [30]:
audio_sim = cosine_sim_audio_df['book']
audio_sim

book                                     1.000000
Still With You                           0.073071
death bed (coffee for your head)         0.001620
Nan-Nan                                  0.020860
Mo-Eh-Wa                                 0.022798
                                           ...   
Murder in the City                       0.617823
September Song - Guitar Acoustic         0.017760
Control                                  0.617787
Heart's Content - Strings Mix            0.617774
We Don't Have To Take Our Clothes Off    0.004434
Name: book, Length: 8521, dtype: float64

# 가사 감정 벡터 - 도서 text 감정 벡터

In [31]:
lyrics_data = data.iloc[:,5:-11]
lyrics_data

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,0.016461,0.265974,0.067839,0.144616,0.116033,0.206217,0.015079,0.134595,0.002582,0.028249,0.002356
1,0.122187,0.059084,0.039108,0.113228,0.024429,0.023013,0.015683,0.212476,0.004940,0.384384,0.001468
2,0.005816,0.248961,0.003242,0.483580,0.044740,0.008447,0.022421,0.152159,0.000405,0.029745,0.000483
3,0.077518,0.249271,0.011364,0.406069,0.011648,0.038887,0.005062,0.123563,0.002235,0.072786,0.001598
4,0.001735,0.235752,0.001410,0.463885,0.142529,0.009176,0.010140,0.120540,0.000858,0.013238,0.000737
...,...,...,...,...,...,...,...,...,...,...,...
8515,0.011245,0.162807,0.006966,0.456847,0.082303,0.040452,0.002504,0.231635,0.000722,0.004143,0.000376
8516,0.021068,0.348507,0.035223,0.432478,0.046740,0.014127,0.007841,0.074129,0.001142,0.017115,0.001629
8517,0.002924,0.170607,0.004646,0.211214,0.003779,0.007508,0.575163,0.020068,0.000557,0.002283,0.001252
8518,0.004582,0.297325,0.004344,0.583962,0.050943,0.004454,0.001543,0.051992,0.000209,0.000479,0.000167


In [32]:
lyrics_data.columns

Index(['empty', 'sadness', 'enthusiasm', 'worry', 'love', 'fun', 'hate',
       'happiness', 'boredom', 'relief', 'anger'],
      dtype='object')

In [33]:
# 도서 감정 벡터를, 가사 감정 벡터 데이터프레임에 이어붙히기
lyrics_data_1 = pd.concat([sentiment_prob, lyrics_data.T],axis=1).T
lyrics_data_1 

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
prob,0.000408,0.062702,0.003605,0.058310,0.794448,0.002706,0.000799,0.060146,0.000238,0.016465,0.000174
0,0.016461,0.265974,0.067839,0.144616,0.116033,0.206217,0.015079,0.134595,0.002582,0.028249,0.002356
1,0.122187,0.059084,0.039108,0.113228,0.024429,0.023013,0.015683,0.212476,0.004940,0.384384,0.001468
2,0.005816,0.248961,0.003242,0.483580,0.044740,0.008447,0.022421,0.152159,0.000405,0.029745,0.000483
3,0.077518,0.249271,0.011364,0.406069,0.011648,0.038887,0.005062,0.123563,0.002235,0.072786,0.001598
...,...,...,...,...,...,...,...,...,...,...,...
8515,0.011245,0.162807,0.006966,0.456847,0.082303,0.040452,0.002504,0.231635,0.000722,0.004143,0.000376
8516,0.021068,0.348507,0.035223,0.432478,0.046740,0.014127,0.007841,0.074129,0.001142,0.017115,0.001629
8517,0.002924,0.170607,0.004646,0.211214,0.003779,0.007508,0.575163,0.020068,0.000557,0.002283,0.001252
8518,0.004582,0.297325,0.004344,0.583962,0.050943,0.004454,0.001543,0.051992,0.000209,0.000479,0.000167


In [34]:
col = ['book']+list(data['name'])

In [35]:
cosine_sim_lyrics = cosine_similarity(lyrics_data_1)
cosine_sim_lyrics_df = pd.DataFrame(cosine_sim_lyrics, index =col, columns=col)
cosine_sim_lyrics_df

,book,Still With You,death bed (coffee for your head),Nan-Nan,Mo-Eh-Wa,YASASHISA,Kiri Ga Naikara,Tsumi No Kaori,Cho Si Noccha Te,Tokuninai,...,My Heart Goes (La Di Da),Back To You,Tick Tack - Extended Mix,Turn Me Up - ViP Mix,Miserable Man,Murder in the City,September Song - Guitar Acoustic,Control,Heart's Content - Strings Mix,We Don't Have To Take Our Clothes Off
book,1.000000,0.381452,0.128534,0.195644,0.141644,0.366686,0.281746,0.114862,0.309151,0.324472,...,0.168130,0.852304,0.968733,0.463718,0.140711,0.266213,0.197058,0.054418,0.182279,0.490150
Still With You,0.381452,1.000000,0.425674,0.701512,0.741244,0.720355,0.736091,0.579968,0.741391,0.806184,...,0.770901,0.681153,0.452202,0.860433,0.645661,0.704713,0.756607,0.340225,0.650681,0.805498
death bed (coffee for your head),0.128534,0.425674,1.000000,0.427589,0.523732,0.384234,0.418146,0.322419,0.463423,0.454470,...,0.289112,0.265163,0.148591,0.486258,0.286708,0.449359,0.362375,0.161462,0.308787,0.410427
Nan-Nan,0.195644,0.701512,0.427589,1.000000,0.978559,0.981930,0.994108,0.970731,0.981328,0.981550,...,0.887594,0.512962,0.388462,0.670581,0.964985,0.971782,0.968385,0.445183,0.979399,0.934484
Mo-Eh-Wa,0.141644,0.741244,0.523732,0.978559,1.000000,0.948702,0.975881,0.945951,0.952311,0.964883,...,0.898202,0.495309,0.327175,0.747536,0.949936,0.937856,0.968041,0.418077,0.956923,0.908138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Murder in the City,0.266213,0.704713,0.449359,0.971782,0.937856,0.962677,0.963903,0.923977,0.988209,0.966096,...,0.785445,0.504854,0.442816,0.689337,0.888574,1.000000,0.898331,0.377529,0.922304,0.913763
September Song - Guitar Acoustic,0.197058,0.756607,0.362375,0.968385,0.968041,0.957601,0.976385,0.939499,0.935785,0.965289,...,0.966941,0.572421,0.380675,0.710821,0.978626,0.898331,1.000000,0.438148,0.974910,0.935146
Control,0.054418,0.340225,0.161462,0.445183,0.418077,0.417850,0.447162,0.431568,0.407786,0.440859,...,0.442076,0.395134,0.137649,0.304154,0.439843,0.377529,0.438148,1.000000,0.420019,0.494162
Heart's Content - Strings Mix,0.182279,0.650681,0.308787,0.979399,0.956923,0.972185,0.980372,0.989085,0.940497,0.947384,...,0.900202,0.496447,0.394319,0.623342,0.995915,0.922304,0.974910,0.420019,1.000000,0.909440


In [36]:
lyrics_sim = cosine_sim_lyrics_df['book']
lyrics_sim

book                                     1.000000
Still With You                           0.381452
death bed (coffee for your head)         0.128534
Nan-Nan                                  0.195644
Mo-Eh-Wa                                 0.141644
                                           ...   
Murder in the City                       0.266213
September Song - Guitar Acoustic         0.197058
Control                                  0.054418
Heart's Content - Strings Mix            0.182279
We Don't Have To Take Our Clothes Off    0.490150
Name: book, Length: 8521, dtype: float64

# 가사 내용 - 도서 text 키워드 유사도

In [37]:
keyword_data = data['key_word']
keyword_data

0       face want see eye tell ecstatic memory even da...
1       stay awake long go bed make cup coffee head ge...
2       say didnt ask anything face w swear corner flo...
3       night blown cold wind dont wander blow inner w...
4       dream dream shade person kill kindness dream s...
                              ...                        
8515    get murder city revenge name one person dead p...
8516    innocent darling use talk til morning mixtape ...
8517    wan na lose control even though old know shake...
8518    love live without u one thing know sure love f...
8519    take clothes good time oh oh could dance party...
Name: key_word, Length: 8520, dtype: object

In [38]:
total_text

'fresh cover manner design base illustration milan kunderas representative work milan kundera live mythborn history deal existence human being surpass history life love four men woman carry czech republic 1960s heavy history personal trauma heavy history shake europe 1970sis draw young teresa work small bar hometown meet tomasi surgeon visit citytomasi burden serious love since divorce ex wife begin live orphan without leave weak charm baby like babyhowever tomas cant throw away light life call erotic friendshipteresa watch tomas struggle pain jealousy resignation czech lose freedom due soviet invasion two go switzerland togetherteresa believe would able escape lover tomasi leave czech republic lose faith tomas constant affair return prague alonethe life two people mixed jealousy hatred gradually add weightmeanwhile tomasis another lover painter sabina want live freely weight homeland historysabina far czech republic farthest enjoy stable daily life love scholar love sabina family head

In [39]:
book_song_cont1 = pd.DataFrame({"text": total_text}, index = range(1))
book_song_cont1

,text
0,fresh cover manner design base illustration mi...


In [40]:
book_song_cont2 = pd.DataFrame({"text": keyword_data})
book_song_cont2

,text
0,face want see eye tell ecstatic memory even da...
1,stay awake long go bed make cup coffee head ge...
2,say didnt ask anything face w swear corner flo...
3,night blown cold wind dont wander blow inner w...
4,dream dream shade person kill kindness dream s...
...,...
8515,get murder city revenge name one person dead p...
8516,innocent darling use talk til morning mixtape ...
8517,wan na lose control even though old know shake...
8518,love live without u one thing know sure love f...


In [41]:
# 전처리를 거친 도서 text를, 전처리를 거친 노래 가사 text 열에 추가
keyword_data_1 = pd.concat([book_song_cont1, book_song_cont2], axis=0).reset_index(drop=True)
keyword_data_1

,text
0,fresh cover manner design base illustration mi...
1,face want see eye tell ecstatic memory even da...
2,stay awake long go bed make cup coffee head ge...
3,say didnt ask anything face w swear corner flo...
4,night blown cold wind dont wander blow inner w...
...,...
8516,get murder city revenge name one person dead p...
8517,innocent darling use talk til morning mixtape ...
8518,wan na lose control even though old know shake...
8519,love live without u one thing know sure love f...


In [42]:
tfidf_vect_cont = TfidfVectorizer()
tfidf_matrix_cont = tfidf_vect_cont.fit_transform(keyword_data_1['text'])
tfidf_array_cont = tfidf_matrix_cont.toarray()
tfidf_df_cont = pd.DataFrame(tfidf_array_cont, columns=tfidf_vect_cont.get_feature_names_out())
tfidf_df_cont

,00,000,00s,01,03,04,050,06,07,085,...,zuckerberg,zum,zumba,zumbe,zun,zunk,zurna,zyaada,zyla,zzoil
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
col = ['book']+list(data['name'])

In [44]:
cosine_sim_keyword = cosine_similarity(tfidf_array_cont)
cosine_sim_keyword_df = pd.DataFrame(cosine_sim_keyword, index = col, columns=col)
cosine_sim_keyword_df

,book,Still With You,death bed (coffee for your head),Nan-Nan,Mo-Eh-Wa,YASASHISA,Kiri Ga Naikara,Tsumi No Kaori,Cho Si Noccha Te,Tokuninai,...,My Heart Goes (La Di Da),Back To You,Tick Tack - Extended Mix,Turn Me Up - ViP Mix,Miserable Man,Murder in the City,September Song - Guitar Acoustic,Control,Heart's Content - Strings Mix,We Don't Have To Take Our Clothes Off
book,1.000000,0.028763,0.018134,0.016621,0.036303,0.029914,0.017143,0.016250,0.025600,0.028438,...,0.009710,0.010650,0.015360,0.009541,0.003868,0.056042,0.003359,0.040108,0.040795,0.014964
Still With You,0.028763,1.000000,0.019357,0.064254,0.069050,0.005153,0.012317,0.031798,0.060540,0.032770,...,0.019688,0.000000,0.012980,0.015875,0.012607,0.042738,0.004408,0.063718,0.006676,0.058046
death bed (coffee for your head),0.018134,0.019357,1.000000,0.000000,0.022849,0.000000,0.000000,0.000000,0.000000,0.014350,...,0.015391,0.096933,0.000000,0.006341,0.000000,0.064764,0.000000,0.011005,0.016866,0.007838
Nan-Nan,0.016621,0.064254,0.000000,1.000000,0.017865,0.060573,0.039396,0.016903,0.020827,0.044412,...,0.012915,0.000000,0.012719,0.015778,0.049240,0.058831,0.000000,0.018042,0.004667,0.011464
Mo-Eh-Wa,0.036303,0.069050,0.022849,0.017865,1.000000,0.055641,0.064496,0.023952,0.033945,0.022338,...,0.059863,0.031588,0.033063,0.008141,0.011345,0.053875,0.003967,0.010416,0.063762,0.027387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Murder in the City,0.056042,0.042738,0.064764,0.058831,0.053875,0.066903,0.013639,0.004625,0.012664,0.036328,...,0.014558,0.061919,0.016869,0.002869,0.016313,1.000000,0.000000,0.038961,0.039023,0.038047
September Song - Guitar Acoustic,0.003359,0.004408,0.000000,0.000000,0.003967,0.000000,0.000000,0.000000,0.000000,0.003927,...,0.000000,0.000000,0.012228,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
Control,0.040108,0.063718,0.011005,0.018042,0.010416,0.039542,0.015868,0.020059,0.005629,0.072138,...,0.006746,0.363678,0.007589,0.005881,0.000000,0.038961,0.000000,1.000000,0.065955,0.148051
Heart's Content - Strings Mix,0.040795,0.006676,0.016866,0.004667,0.063762,0.024154,0.001346,0.007473,0.020818,0.015582,...,0.006187,0.000000,0.003826,0.000000,0.003795,0.039023,0.000000,0.065955,1.000000,0.167936


In [45]:
keyword_sim = cosine_sim_keyword_df['book']
keyword_sim

book                                     1.000000
Still With You                           0.028763
death bed (coffee for your head)         0.018134
Nan-Nan                                  0.016621
Mo-Eh-Wa                                 0.036303
                                           ...   
Murder in the City                       0.056042
September Song - Guitar Acoustic         0.003359
Control                                  0.040108
Heart's Content - Strings Mix            0.040795
We Don't Have To Take Our Clothes Off    0.014964
Name: book, Length: 8521, dtype: float64

# 전체 유사도 계산

In [46]:
total_sim  = 0.8*audio_sim + 0.1*lyrics_sim + 0.1*keyword_sim

recommend_song = total_sim.sort_values(ascending=False)[1:6].index
total_sim_df = pd.DataFrame(total_sim[1:])
total_sim_df = total_sim_df.reset_index()
total_sim_df.columns = ['name','book']

top_five = total_sim_df.sort_values(by='book',ascending=False)[:5]
index = total_sim_df.sort_values(by='book',ascending=False)[:5].index.sort_values()

In [47]:
artist = data.iloc[index][['url','name','Artist']]
top_five_df = pd.merge(artist,top_five,on='name').sort_values(by='book',ascending=False).drop_duplicates()
top_five_df 

,url,name,Artist,book
3,https://open.spotify.com/track/4XZDsW1GTO6J59D...,Larga essa Rua - Ao Vivo,Gusttavo Lima,0.737128
1,https://open.spotify.com/track/1lORkxEMmsCZqho...,Nothing,Bruno Major,0.660078
0,https://open.spotify.com/track/4rvQ7BtQl20UuLm...,Eslabon Intro,Eslabon Armado,0.651045
4,https://open.spotify.com/track/1d5lpW3gUyq537i...,Samjho Na,Aditya Rikhari,0.633349
2,https://open.spotify.com/track/63Zaf8Rci6eOUPq...,La Mitad,Camilo,0.632147


In [48]:
total_sim_2  = 0*audio_sim + 0.5*lyrics_sim + 0.5*keyword_sim

recommend_song = total_sim_2.sort_values(ascending=False)[1:6].index
total_sim_df_2 = pd.DataFrame(total_sim_2[1:])
total_sim_df_2 = total_sim_df_2.reset_index()
total_sim_df_2.columns = ['name','book']

top_five_2 = total_sim_df_2.sort_values(by='book',ascending=False)[:5]
index = total_sim_df_2.sort_values(by='book',ascending=False)[:5].index.sort_values()

In [49]:
artist = data.iloc[index][['url','name','Artist']]
top_five_df_2 = pd.merge(artist,top_five_2,on='name').sort_values(by='book',ascending=False).drop_duplicates()
top_five_df_2

,url,name,Artist,book
3,https://open.spotify.com/track/12g9IeQzX7xECLN...,I'm In Love With You,The 1975,0.585494
0,https://open.spotify.com/track/7JPqgWtxGyXmuFp...,Amor Eterno,Los Del Limit,0.580851
1,https://open.spotify.com/track/4ZhVzE3rSxnJLqf...,Too Much To Ask,Don Diablo,0.578922
4,https://open.spotify.com/track/1iKE95R4d2eSJZV...,Por Ti,Various Artists,0.576399
2,https://open.spotify.com/track/3T7qQEncNHkrBGF...,Everything Happens For A Reason,Madison Beer,0.575675


In [53]:
url_list = top_five_df['url']
url_list

3    https://open.spotify.com/track/4XZDsW1GTO6J59D...
1    https://open.spotify.com/track/1lORkxEMmsCZqho...
0    https://open.spotify.com/track/4rvQ7BtQl20UuLm...
4    https://open.spotify.com/track/1d5lpW3gUyq537i...
2    https://open.spotify.com/track/63Zaf8Rci6eOUPq...
Name: url, dtype: object

In [54]:
top_five_df.iloc[1]

url       https://open.spotify.com/track/1lORkxEMmsCZqho...
name                                                Nothing
Artist                                          Bruno Major
book                                               0.660078
Name: 1, dtype: object

In [56]:
print('분위기 유사도 TOP 1\n')
print('제목 : {0} \n가수 : {1} \nurl : {2} \n유사도 : {3:.4f}'.format(top_five_df.iloc[0]['name'],top_five_df.iloc[0]['Artist'],top_five_df.iloc[0]['url'],top_five_df.iloc[0]['book']))

분위기 유사도 TOP 1

제목 : Larga essa Rua - Ao Vivo 
가수 : Gusttavo Lima 
url : https://open.spotify.com/track/4XZDsW1GTO6J59DAoIpPYB 
유사도 : 0.7371


In [57]:
print('내용 유사도 TOP 1\n')
print('제목 : {0} \n가수 : {1} \nurl : {2} \n유사도 : {3:.4f}'.format(top_five_df_2.iloc[0]['name'],top_five_df_2.iloc[0]['Artist'],top_five_df_2.iloc[0]['url'],top_five_df_2.iloc[0]['book']))

내용 유사도 TOP 1

제목 : I'm In Love With You 
가수 : The 1975 
url : https://open.spotify.com/track/12g9IeQzX7xECLNxz8dpb5 
유사도 : 0.5855
